# Running RBAtools for RBA variability analysis

## Import packages

In [ ]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import rba
import pickle

from ipywidgets import IntProgress
from IPython.display import display

from rbatools.rba_session import SessionRBA

## Define functions

In [ ]:
# function to record only protein results
# based on SessionRBA method record_results
import rbatools._auxiliary_functions as _auxiliary_functions

def rec_prots(sessionRBA, run_name: str):
    if not hasattr(sessionRBA, 'Results'):
            sessionRBA.Results = {'Reactions': pd.DataFrame(index=list(sessionRBA.ModelStructure.ReactionInfo.Elements.keys())),
                            'Enzymes': pd.DataFrame(index=list(sessionRBA.ModelStructure.EnzymeInfo.Elements.keys())),
                            'Processes': pd.DataFrame(index=[sessionRBA.ModelStructure.ProcessInfo.Elements[i]['ID']+'_machinery' for i in sessionRBA.ModelStructure.ProcessInfo.Elements.keys()]),
                            'Proteins': pd.DataFrame(index=list(sessionRBA.ModelStructure.ProteinMatrix['Proteins'])),
                            'ProtoProteins': pd.DataFrame(index=list(sessionRBA.ModelStructure.ProteinGeneMatrix['ProtoProteins'])),
                            'Constraints': pd.DataFrame(index=sessionRBA.Problem.LP.row_names),
                            'SolutionType': pd.DataFrame(index=['SolutionType']),
                            'ObjectiveFunction': pd.DataFrame(index=sessionRBA.Problem.LP.col_names),
                            'Mu': pd.DataFrame(index=['Mu']),
                            'ObjectiveValue': pd.DataFrame(index=['ObjectiveValue']),
                            'ExchangeFluxes': pd.DataFrame(index=list(sessionRBA.ExchangeMap.keys()))}
    
    sessionRBA.Results['Proteins'][run_name] = _auxiliary_functions.record_proteome(RBA_Session=sessionRBA, run=run_name)

## load the model

In [ ]:
xml_dir = 'model\\'
output_dir = 'RBVA_MJahn_model\\'
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

simulation = SessionRBA(xml_dir, lp_solver = 'cplex')

In [ ]:
substrate = pd.read_csv('simulation\\substrate_input.csv')
# print(substrate)
mediumDict = simulation.get_medium()

# select medium composition for analysis
mediumRow = 2 # 0 to 2
sessName = '240115_RBVA'
if mediumRow == 0:
    sessName = sessName + '_80mMFA_fullN'
elif mediumRow == 1:
    sessName = sessName + '_80mMFA_limN'
elif mediumRow == 2:
    sessName = sessName + '_150mMFA_limN'

In [ ]:
# print(mediumDict)
print(float(substrate['carbon_conc'].loc[mediumRow]))
print(float(substrate['nitrogen_conc'].loc[mediumRow]))

### set medium

In [ ]:
carbonConc = float(substrate['carbon_conc'].loc[mediumRow])
nitrogenConc = float(substrate['nitrogen_conc'].loc[mediumRow])

simulation.set_medium({'M_for': carbonConc})
simulation.set_medium({'M_nh4': nitrogenConc})
print(mediumDict)

### get reactions and mumax

In [ ]:
# get list of all proteins to later relate IDs to proteomics data

proteinList = simulation.get_proteins()
proteinInformation = {}
for prot in proteinList:
    protInformation = simulation.get_protein_information(protein = prot)
    proteinInformation[prot] = protInformation

# # export this dictionary to later map reaction names to proper identifiers
saveFileName = 'C_necator_MJahn_RBA_model_proteins'
curFolder = os.getcwd()
savePath = os.path.join(curFolder, saveFileName + '.pkl')

with open(savePath, 'wb+') as f:
    pickle.dump(proteinInformation, f)

In [ ]:
reactionList = simulation.get_reactions()

mumax = simulation.find_max_growth_rate()

## Record simple RBA result

In [ ]:
print(mumax)
simulation.record_results(run_name="mu_max")
simulation.write_results(session_name = sessName + '_muMax')

In [ ]:
simulation.SimulationData.export_csv(output_directory = output_dir + 'manual_RBVA_results\\')

## Simulate variability of each reaction

In [ ]:
simulation.set_growth_rate(Mu = mumax)

for rxn in reactionList:
    simulation.Problem.clear_objective()
    simulation.Problem.set_objective(inputDict = {rxn: 1.0})
    simulation.Problem.solve_lp(feasible_stati=["optimal","feasible","feasible_only_before_unscaling"],try_unscaling_if_sol_status_is_feasible_only_before_unscaling=False)
#     simulation.record_results(run_name = rxn + '_min')
    rec_prots(simulation, run_name = rxn + '_min')
# edit the record_results funtion to only record the proteome
    simulation.Problem.set_objective(inputDict = {rxn: -1.0})
    simulation.Problem.solve_lp(feasible_stati=["optimal","feasible","feasible_only_before_unscaling"],try_unscaling_if_sol_status_is_feasible_only_before_unscaling=False)
#     simulation.record_results(run_name = rxn + '_max')
    rec_prots(simulation, run_name = rxn + '_max')

simulation.write_results(session_name = sessName) 
simulation.SimulationData.export_csv(output_directory = output_dir + 'manual_RBVA_results\\')

In [ ]:
simulation.write_results(session_name = sessName) 
simulation.SimulationData.export_csv(output_directory = output_dir + 'manual_RBVA_results\\')

save the feasible ranges as a dictionary

In [ ]:
saveFileName = '240105_RBVA_ranges_150mM_FA_limN'
curFolder = os.getcwd()
savePath = os.path.join(curFolder, 'simulation', 'RBVA_results', saveFileName + '.pkl')

with open(savePath, 'wb+') as f:
    pickle.dump(Feasible_Ranges, f)